<a href="https://colab.research.google.com/github/YoruAlptraum/video-downloader/blob/main/save_playlist_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp
!pip install firebase-admin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.6 MB/s eta 0:00:00


In [3]:
from yt_dlp import YoutubeDL
from datetime import date
import json
import os
import re

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# firebase/firestore

# Uploaded service account key file
cred = credentials.Certificate(userdata.get('FIREBASE_CERT_PATH'))

# Initialize the Firebase Admin SDK if it hasn't been initialized yet
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

# Get a Firestore client
db = firestore.client()

print("Firebase Admin SDK initialized and Firestore client obtained!")

Firebase Admin SDK initialized and Firestore client obtained!


In [5]:
favorite_music_url = 'https://www.youtube.com/playlist?list=PL2E-fKOunybIoCAccVG2flYZSmltOymBa'
playlist_url = 'https://www.youtube.com/playlist?list=PL2E-fKOunybLmPf--AqaUFZVVtrfIpNXi'
col = db.collection('playlist')

In [ ]:
def get_playlist_entries(playlist_url):
    # get info on playlist
    # ℹ️ See help(YoutubeDL) for a list of available options and public functions
    ydl_opts = {'extract_flat': True}
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(playlist_url, download=False)

        # ℹ️ ydl.sanitize_info makes the info json-serializable
        info = json.dumps(ydl.sanitize_info(info))

    info_dict = json.loads(info)
    entries = info_dict['entries']

    return entries

In [ ]:
def save_playlist_data():
    playlist = get_playlist_entries(playlist_url)
    favorites = get_playlist_entries(favorite_music_url)
    favorites = set([i['id'] for i in favorites])
    today = date.today().strftime("%d/%m/%Y")

    for video in playlist:
        video_id = video['id']
        # check if on favorites list
        favorite = video_id in favorites

        # check if already on list
        doc = col.document(video_id).get()

        if doc.exists:
            # if on list check if video became unavailable
            if video['duration'] == None:
                col.document(video_id).update({'removedFromYT': today})
                col.document(video_id).update({'lastUpdated': today})
            else:
                col.document(video_id).update({'lastUpdated': today})
        else:
            # skip if the video info cannot be salvaged
            if video['duration'] == None:
                continue

            data = {
                'videoId': video_id,
                'title': video['title'],
                'channel': video['channel'],
                'favorite': favorite,

                'addedToList': today,
                'lastUpdated': today,

                'removedFromYT': None,
            }

            # add to list
            col.document(video_id).set(data)

    return

In [ ]:
save_playlist_data()

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PL2E-fKOunybLmPf--AqaUFZVVtrfIpNXi
[youtube:tab] PL2E-fKOunybLmPf--AqaUFZVVtrfIpNXi: Downloading webpage
[youtube:tab] PL2E-fKOunybLmPf--AqaUFZVVtrfIpNXi: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Playlist
[youtube:tab] PL2E-fKOunybLmPf--AqaUFZVVtrfIpNXi page 1: Downloading API JSON
[youtube:tab] PL2E-fKOunybLmPf--AqaUFZVVtrfIpNXi page 2: Downloading API JSON
[youtube:tab] Playlist Playlist: Downloading 298 items of 298
[download] Downloading item 1 of 298
[download] Downloading item 2 of 298
[download] Downloading item 3 of 298
[download] Downloading item 4 of 298
[download] Downloading item 5 of 298
[download] Downloading item 6 of 298
[download] Downloading item 7 of 298
[download] Downloading item 8 of 298
[download] Downloading item 9 of 298
[download] Downloading item 10 of 298
[download] Downloading item 11 of 298
[download] Downloading item 12 of 298
[download] Dow

In [20]:
docs = col.stream()
i = 0
for doc in docs:
    if doc.to_dict().get('favorite'):
        i += 1
        print(f' {i} {doc.to_dict().get("title")}')

 1 [MV] Red - Calliope Mori #HololiveEnglish #HoloMyth
 2 Roe Kapara & Indy June – Daisies (Prod. Sensu) (Lyrics) [CC]
 3 [Zombie Land Saga] Franchouchou "Saga Jihen" / Hoshimachi Suisei with Hololive Fantasy
 4 light switch - miku cover
 5 「Nightcore」→ Don't Panic
 6 Gawr Gura- The Moon Song [EDIT]
 7 Gawr Gura Ai Song cover Sweater Weather
 8 Tevin Campbell - Can We Talk (Official Video)
 9 comet/Hoshimati Suisei
 10 this is what falling in love feels like (Multilingual)
 11 Circles
 12 flos / 猫又おかゆ (Cover)
 13 SHIGURE UI x RYAN GOSLING (9MM)
 14 Ariana Grande - thank u, next (audio)
 15 Nappy Roots - Here We Go Again
 16 Arknights EP - Code of Brawl [Speed of Light] DJ OKAWARI feat. Ai Ninomiya
 17 Beautiful scene in anime [FHD]
 18 Think Of Me Once In A While, Take Care
 19 [ Gawr Gura ] Heather - Conan Gray (Ai Cover)
 20 When I'm Gone - Gura x Senzawa Cover Duet
 21 Hoshimachi Suisei - BIBBIDIBA (official)
 22 Kiss Of Life
 23 【original anime MV】SHINKIRO【hololive/宝鐘マリン・Gawr Gura】